# Evaluates the feasibility of using the H3 library for finding neighbors

In [2]:
import pandas as pd
from shapely import wkt
import h3
from tqdm import tqdm

junctions_csv = "junctionsdf_stuttgart.csv"
h3_resolution = 8

In [10]:
# Below copied from clusterJcts.py:

def largeIntersection(poly1, poly2):
    if not (poly1.intersects(poly2)):
        return False
    elif (((poly1.intersection(poly2).area/poly1.area)*100) > 8):
        return True
    elif (((poly1.intersection(poly2).area/poly2.area)*100) > 8):
        return True

# c) Check if two junctions share a square (which means they should end up in the same cluster)
def sharedSquare(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    if lst3 != []:
        for elem in lst3:
            if 'platz' in elem or 'Platz' in elem:
                return True
    return False

# c) Put it all together to assign its neighbours to each junction

def neighbourFindingWrapper(junctionsdf):

    # Use buffer trick if polygon is invalid
    # https://stackoverflow.com/questions/13062334/polygon-intersection-error-in-shapely-shapely-geos-topologicalerror-the-opera

    junctionsdf['poly_geometry'] = junctionsdf['poly_geometry'].map(lambda poly: poly if poly.is_valid else poly.buffer(0))

    ops_number = junctionsdf.index.size
    # add row number, needed to check only x rows above/below current row in getNeighbours
    junctionsdf['row_number'] = [x for x in range(0, ops_number)]
    neighbours_list = []
    bar = tqdm(total=ops_number, desc="Computing Neighbours")

    # prepare lists outside of loop to get rid of expensive pandas operations
    index_list = junctionsdf.index.tolist()
    poly_list = junctionsdf['poly_geometry'].tolist()
    highway_list = junctionsdf['highwaynames'].tolist()

    for id, geometry, highway_name, row_number in zip(junctionsdf.index,junctionsdf['poly_geometry'],junctionsdf['highwaynames'],junctionsdf['row_number']):
        neighbours_list.append(getNeighbours(id, geometry, highway_name, row_number, index_list, poly_list, highway_list))
        bar.update(1)

    # remove row number
    junctionsdf.drop(columns=['row_number'], inplace=True)
    junctionsdf['neighbours'] = neighbours_list
    bar.close()

    # junctionsdf.dropna(subset=['neighbours'], inplace=True)

    return junctionsdf

def getNeighbours(outerInd, outerPoly, outerHighways, row_number, index_list, poly_list, highway_list):
    max_row_diff = 1500
    # we do not need to check all rows for neighbours. to be safe, let's check max_row_diff above and max_row_diff below row number
    lower_range = max(0, row_number - max_row_diff)
    upper_range = min(len(index_list), row_number + max_row_diff)

    # check largeIntersection or sharedSquare for each item in poly_list/highway_list
    neighbours = []
    for i in range(lower_range, upper_range):
        if outerInd == index_list[i]:
            continue

        intersection = largeIntersection(poly_list[i], outerPoly)
        square = sharedSquare(highway_list[i], outerHighways)

        if intersection or square:
            diff_percentage = abs(row_number - i) / float(max_row_diff)
            if diff_percentage >= 0.8:
                print("Row difference at {0!s}%, absolute max_row_diff is {1!s}".format(diff_percentage, abs(row_number - i)))

            neighbours.append(index_list[i])

    return neighbours

In [11]:
# read in csv file that comprises the junctionsdf available also in clussterJcts.py

df = pd.read_csv(junctions_csv, index_col="id")

# parse string poly
df['poly_geometry'] = df['poly_geometry'].apply(wkt.loads)

# ignore invalid polys as done in clusterJcts.py
df['poly_geometry'] = df['poly_geometry'].map(lambda poly: poly if poly.is_valid else poly.buffer(0))

df.head()

Computing Neighbours:   0%|          | 1/66448 [11:36<12860:00:38, 696.74s/it]


,lat,lon,highwayids,highwaynames,highwaytypes,highwaylanes,highwaylanesBw,poly_geometry,poly_vertices_lats,poly_vertices_lons
id,,,,,,,,,,
173097876,48.520746,8.775702,"[36731756, 372332684, 426974536]","['K 6940', 'L 1361', 'L 1361']","['tertiary', 'secondary', 'secondary']","['unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown']","POLYGON ((48.52090537488439 8.775698510692514,...","array('d', [48.52090537488439, 48.520743351014...","array('d', [8.775698510692514, 8.7754626614868..."
426938252,48.522502,8.777334,"[27120149, 36731756]","['K 1072', 'K 6940']","['tertiary', 'tertiary']","['unknown', 'unknown']","['unknown', 'unknown']","POLYGON ((48.52265266750056 8.777330337629358,...","array('d', [48.52265266750056, 48.522499905509...","array('d', [8.777330337629358, 8.7771079528511..."
173096469,48.532423,8.775203,"[16743795, 16743802, 23577078]","['Brühlstraße', 'Brühlstraße', 'Schlossgartens...","['tertiary', 'tertiary', 'tertiary']","['unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown']","POLYGON ((48.53257366679242 8.775198930130973,...","array('d', [48.532573666792416, 48.53242090109...","array('d', [8.775198930130973, 8.7749765082359..."
173096563,48.531917,8.773936,"[16743802, 26934398, 16743810]","['Brühlstraße', 'Vollmaringer Straße', 'K 1072']","['tertiary', 'residential', 'tertiary']","['unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown']","POLYGON ((48.53207222041966 8.773932315071427,...","array('d', [48.53207222041966, 48.531914822815...","array('d', [8.773932315071427, 8.7737031593739..."
281413533,48.529614,8.774486,"[25799537, 27119682, 158188075, 16743810]","['Goethestraße', 'In der Röte', 'Baisinger Str...","['residential', 'residential', 'tertiary', 'te...","['unknown', 'unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown', 'unknown']","POLYGON ((48.52977149741602 8.774481761402312,...","array('d', [48.52977149741602, 48.529611786151...","array('d', [8.774481761402312, 8.7742492446035..."


In [12]:
# this is just code to experiment with h3

# select row by index
row = df.loc[173097876]

# do some h3 operations on lat/lon of row
h3_str = h3.geo_to_h3(row["lat"], row["lon"], h3_resolution)
h3_cell = h3.h3_get_base_cell(h3_str)
h3_ring = h3.hex_ring(h3_str, 1)

print("{0!s}/{1!s} has the h3 string {2!s} in the cell {3!s}".format(row["lat"], row["lon"], h3_str, h3_cell))
print("The h3 ring comprises {0!s}".format(h3_ring))

48.520746/8.7757024 has the h3 string 881f81b1e1fffff in the cell 15
The h3 ring comprises {'881f81b1edfffff', '881f81b1e9fffff', '881f81b1e7fffff', '881f81b1ebfffff', '881f81b1e3fffff', '881f81b1e5fffff'}


## Below starts the actual code that does the neighbour search with H3

In [13]:
# add an h3 column
df["h3"] = df.apply (lambda row: h3.geo_to_h3(row["lat"], row["lon"], h3_resolution), axis=1)
df["h3"].describe()

count               66448
unique               4054
top       881f8c9001fffff
freq                  115
Name: h3, dtype: object

In [5]:
ops_number = df.index.size

bar = tqdm(total=ops_number, desc="Computing Neighbours")
df_h3_only = pd.DataFrame(df.h3.tolist(), index=df.index)
neighbours_list = []

checked = 0

for id in df.index:

    row = df.loc[id]
    h3_ring = h3.hex_ring(row["h3"], 1)
    h3_ring.add(row["h3"]) # add own cell to ring

    # From https://stackoverflow.com/questions/53342715/pandas-dataframe-select-rows-where-a-list-column-contains-any-of-a-list-of-strin
    rows_to_check = df[df_h3_only.isin(h3_ring).any(1).values]
    checked += rows_to_check.size

    neighbour_rows = rows_to_check[rows_to_check.apply(lambda apply_row: (largeIntersection(apply_row['poly_geometry'],row["poly_geometry"]) or sharedSquare(apply_row['highwaynames'], row['highwaynames'])), axis=1)]

    # Grab indices of those rows that passed the filter
    neighbours = neighbour_rows.index.tolist()
    # remove use from the neighbourslist
    neighbours.remove(id)

    neighbours_list.append(neighbours)
    bar.update(1)

print("Checked on average {0!s} entries for neighbours".format(int(checked / df.index.size)))

# remove row number
df['neighbours'] = neighbours_list
bar.close()

NameError: name 'df' is not defined

In [15]:
df['neighbours'].head(50)

id
173097876                         []
426938252                         []
173096469                         []
173096563                [295137117]
281413533                         []
281413535                         []
295133819                         []
295137117     [173096563, 297554382]
297542129                         []
297542135                [297550504]
297545389                         []
297545450                         []
297545470                         []
297550504                [297542135]
297550517                         []
297554378                         []
297554382                [295137117]
281413536                         []
281413538                         []
281413547                         []
281413550                         []
286463745                         []
286463767                [295121658]
290794553                         []
295121649                [295123713]
295121657                         []
295121658                [286463767

## Run old code for comparison

In [16]:
# read in csv file that comprises the junctionsdf available also in clussterJcts.py

df_old = pd.read_csv(junctions_csv, index_col="id")

# parse string poly
df_old['poly_geometry'] = df_old['poly_geometry'].apply(wkt.loads)

# ignore invalid polys as done in clusterJcts.py
df_old['poly_geometry'] = df_old['poly_geometry'].map(lambda poly: poly if poly.is_valid else poly.buffer(0))

df_old.head()

,lat,lon,highwayids,highwaynames,highwaytypes,highwaylanes,highwaylanesBw,poly_geometry,poly_vertices_lats,poly_vertices_lons
id,,,,,,,,,,
173097876,48.520746,8.775702,"[36731756, 372332684, 426974536]","['K 6940', 'L 1361', 'L 1361']","['tertiary', 'secondary', 'secondary']","['unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown']","POLYGON ((48.52090537488439 8.775698510692514,...","array('d', [48.52090537488439, 48.520743351014...","array('d', [8.775698510692514, 8.7754626614868..."
426938252,48.522502,8.777334,"[27120149, 36731756]","['K 1072', 'K 6940']","['tertiary', 'tertiary']","['unknown', 'unknown']","['unknown', 'unknown']","POLYGON ((48.52265266750056 8.777330337629358,...","array('d', [48.52265266750056, 48.522499905509...","array('d', [8.777330337629358, 8.7771079528511..."
173096469,48.532423,8.775203,"[16743795, 16743802, 23577078]","['Brühlstraße', 'Brühlstraße', 'Schlossgartens...","['tertiary', 'tertiary', 'tertiary']","['unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown']","POLYGON ((48.53257366679242 8.775198930130973,...","array('d', [48.532573666792416, 48.53242090109...","array('d', [8.775198930130973, 8.7749765082359..."
173096563,48.531917,8.773936,"[16743802, 26934398, 16743810]","['Brühlstraße', 'Vollmaringer Straße', 'K 1072']","['tertiary', 'residential', 'tertiary']","['unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown']","POLYGON ((48.53207222041966 8.773932315071427,...","array('d', [48.53207222041966, 48.531914822815...","array('d', [8.773932315071427, 8.7737031593739..."
281413533,48.529614,8.774486,"[25799537, 27119682, 158188075, 16743810]","['Goethestraße', 'In der Röte', 'Baisinger Str...","['residential', 'residential', 'tertiary', 'te...","['unknown', 'unknown', 'unknown', 'unknown']","['unknown', 'unknown', 'unknown', 'unknown']","POLYGON ((48.52977149741602 8.774481761402312,...","array('d', [48.52977149741602, 48.529611786151...","array('d', [8.774481761402312, 8.7742492446035..."


In [17]:
df_old = neighbourFindingWrapper(df_old)

Computing Neighbours: 100%|██████████| 66448/66448 [26:02<00:00, 42.54it/s]


Row difference at 0.8786666666666667%, absolute max_row_diff is 1318
Row difference at 0.8786666666666667%, absolute max_row_diff is 1318
Row difference at 0.8706666666666667%, absolute max_row_diff is 1306
Row difference at 0.8606666666666667%, absolute max_row_diff is 1291
Row difference at 0.8706666666666667%, absolute max_row_diff is 1306
Row difference at 0.8606666666666667%, absolute max_row_diff is 1291


In [21]:
df_old["neighbours"].head(50)

id
173097876                         []
426938252                         []
173096469                         []
173096563                [295137117]
281413533                         []
281413535                         []
295133819                         []
295137117     [173096563, 297554382]
297542129                         []
297542135                [297550504]
297545389                         []
297545450                         []
297545470                         []
297550504                [297542135]
297550517                         []
297554378                         []
297554382                [295137117]
281413536                         []
281413538                         []
281413547                         []
281413550                         []
286463745                         []
286463767                [295121658]
290794553                         []
295121649                [295123713]
295121657                         []
295121658                [286463767

In [27]:
# find difference
print("Self = H3, Other = Old Strategy")
df["neighbours"].compare(df_old["neighbours"])

Self = H3, Other = Old Strategy


,self,other
id,,
518990598,[518990600],[]
235605158,"[248095255, 248095256]",[248095255]
248095255,"[235605158, 248095256]",[235605158]
375679972,[684149401],[]
684149401,[375679972],[]
248095256,"[235605158, 248095255, 149033927]",[149033927]
102984577,[646883],[]
20850506,"[82770965, 1580857799, 18240910]","[82770965, 1580857799]"
279006817,[28249736],[]


## Feasibility test for segments

Using h3 for classifying street segments is a little more challenging than it is for junctions as junctions basically are just pairs of lat/lon pairs whereas segments are more complex geometric objects (non-straight lines resp. polygons of varying length, area etc) and h3 is only designed for computing geospatial indices from lat/lon pairs (as far as I can tell).

Sure, a possible approach would be to map every lat/lon pair contained in the node_list associated with a segment onto its geospatial h3 index and then check for set intersections between rows, but that would actually just correspond to almost the exact operation that is carried out for determining if two street segments are neighbours: two street segments are considered neighbours if their node_lists share nodes that aren't junctions. Thus, no efficiency gain from using h3 could be attained using this strategy.

What appears more reasonable is to compute h3 indices from the centroids of the segment's polygons and then use a coarser granularity as with the junctions to account for their generally larger and more complex shapes.

In [114]:
segs4h3eval = pd.read_pickle("segs4h3eval")

In [115]:
segs4h3eval

,highwayname,highwaytype,highwaylanes,lanes:backward,segment_nodes_ids,id,coords,lats,lons,seg_length,poly_geometry,poly_vertices_lats,poly_vertices_lons,oddball,row_number
0,Eutritzscher Straße,secondary,3,unknown,"[26008874, 4909369840, 6127283268, 4909369846]",[1],"((51.3483918, 51.3485208, 51.3485397, 51.34856...","[51.3483918, 51.3485208, 51.3485397, 51.3485657]","[12.3765054, 12.3764885, 12.376486, 12.3764826]",14.400105,"POLYGON ((51.34852 12.37655, 51.34862 12.37653...","[51.34851740052531, 51.3486152808835, 51.34862...","[12.376545401842842, 12.376532573612891, 12.37...",True,0
1,Am Gothischen Bad,primary,2,unknown,"[412129790, 3902411269, 412129799, 1730407499]",[2],"((51.3518231, 51.3517776, 51.3517468, 51.35172...","[51.3518231, 51.3517776, 51.3517468, 51.3517248]","[12.3917171, 12.3917244, 12.3917356, 12.3917496]",5.087584,"POLYGON ((51.35178 12.39166, 51.35164 12.39173...","[51.35177984977018, 51.35164390523494, 51.3516...","[12.391658242402752, 12.39172995908536, 12.391...",True,1
2,Am Gothischen Bad,primary,2,unknown,"[1730407499, 412129805]",[3],"((51.3517248, 51.3516806), (12.3917496, 12.391...","[51.3517248, 51.3516806]","[12.3917496, 12.3917983]",5.974282,"POLYGON ((51.35164 12.39176, 51.35167 12.39189...","[51.35164044910517, 51.35167356252553, 51.3518...","[12.391756208262041, 12.391892361044935, 12.39...",True,2
3,Brandenburger Straße,primary,2,unknown,"[496115658, 18345421, 412129865, 412129864, 18...",[4],"((51.348427, 51.3478659, 51.3477671, 51.347660...","[51.348427, 51.3478659, 51.3477671, 51.3476605...","[12.3925414, 12.3916911, 12.3915613, 12.391470...",86.060609,"POLYGON ((51.34792 12.39165, 51.34770 12.39141...","[51.347918584047235, 51.347696746185015, 51.34...","[12.391646007676021, 12.391410258031101, 12.39...",True,3
4,Brandenburger Straße,primary,2,unknown,"[18345487, 264977989]",[5],"((51.3474152, 51.3474165), (12.3913854, 12.391...","[51.3474152, 51.3474165]","[12.3913854, 12.391304]",5.673157,"POLYGON ((51.34748 12.39129, 51.34736 12.39125...","[51.347478448988184, 51.34735560419748, 51.347...","[12.391292779650358, 12.391249254969415, 12.39...",True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,unknown,cycleway,unknown,unknown,"[276448498, 17086649]",[2885],"((51.3377576, 51.337749), (12.389766, 12.39013...","[51.3377576, 51.337749]","[12.389766, 12.3901392]",26.024707,"POLYGON ((51.33774 12.39014, 51.33776 12.39015...","[51.33774015168444, 51.33775763097183, 51.3377...","[12.390140717912143, 12.390147091016967, 12.38...",True,1867
1868,unknown,cycleway,unknown,unknown,"[2592701510, 6855296732]",[2886],"((51.3388698, 51.3388206), (12.3948754, 12.394...","[51.3388698, 51.3388206]","[12.3948754, 12.3947686]",9.238543,"POLYGON ((51.33883 12.39476, 51.33881 12.39477...","[51.33882867836409, 51.338808904362814, 51.338...","[12.3947631434032, 12.394766181142337, 12.3948...",True,1868
1869,unknown,tertiary_link,1,unknown,"[264903795, 4398887761, 2460220923, 26819450]",[2887],"((51.332166, 51.3321037, 51.3320296, 51.331898...","[51.332166, 51.3321037, 51.3320296, 51.3318983]","[12.39466, 12.3946489, 12.3946217, 12.3945559]",6.974203,"POLYGON ((51.33212 12.39460, 51.33189 12.39449...","[51.33212162129953, 51.33188572956259, 51.3318...","[12.394602246044151, 12.394493331542789, 12.39...",True,1869
1870,unknown,pedestrian,unknown,unknown,"[5494594962, 7258000216]",[2888],"((51.344351, 51.3441141), (12.3835181, 12.3840...","[51.344351, 51.3441141]","[12.3835181, 12.3840913]",47.851482,"POLYGON ((51.34407 12.38408, 51.34414 12.38416...","[51.344066185579884, 51.34413911758109, 51.344...","[12.384076526642737, 12.384161450970716, 12.38...",True,1870


Get lat and lon of the polygon centroids

In [116]:
segs4h3eval['centroid_lat'] = segs4h3eval['poly_geometry'].map(lambda x: x.centroid.x)

In [117]:
segs4h3eval['centroid_lon'] = segs4h3eval['poly_geometry'].map(lambda x: x.centroid.y)

Try it with the junctions' resolution for evaluation

In [118]:
h3_resolution = 8

In [119]:
# add an h3 column
segs4h3eval["h3"] = segs4h3eval.apply (lambda row: h3.geo_to_h3(row["centroid_lat"], row["centroid_lon"], h3_resolution), axis=1)

Read in the df containing the region's junctions (when searching for segments' neighbours, the presence of common nodes that aren't junctions is checked; thus, information on junction nodes is required)

In [120]:
junctions_for_segs = "leipzig_junctions_for_segs.csv"

In [121]:
aux_junctions_df = pd.read_csv(junctions_for_segs)

(a) Neighbour search with h3

In [122]:
def findNeighboursH3(unfoldedOddballs, junctionsdf):

    # these are ALL junctions (i.e., intersections of at least two highways, irrespective of their type)

    jctids = junctionsdf['id'].values

    # now we also need the LARGER junctions (i.e., intersections of at least two highways of a larger type)

    larger_jcts = junctionsdf[junctionsdf['junction'] == 'large_junction']

    larger_jctids = larger_jcts['id'].values 

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Determine neighbours based on shared nodes that aren't junctions.

    def getNeighbours(outerNodes, outerHighwayType, outerInd, row_h3):

        h3_disk = h3.k_ring(row_h3, 1)
        
        # Slice according to 'h3 index in h3_disk'
        df_slice = unfoldedOddballs[pd.DataFrame(unfoldedOddballs.h3.tolist()).isin(h3_disk).any(1).values]
        
        print(len(df_slice))
        
        # prepare lists outside of map operations to get rid of expensive pandas operations
        segment_nodes_list = df_slice['segment_nodes_ids'].tolist()
        index_list = df_slice.index.tolist()

        # Filter the 'segment_nodes_ids' column so that only those elements that are also contained in outerNodes
        # remain
        
        common_nodes = map(lambda innerNodes: set(innerNodes).intersection(set(outerNodes)), segment_nodes_list)

        # Convert back to list (from set)

        common_nodes_list = map(lambda x: list(x), common_nodes)

        # If we're looking at smaller highway types, check if any of the nodes each row has in common with outerNodes
        # is a junction of any type (small or large, the last one meaning that at least two highways of a larger
        # type - residential, primary, trunk, etc - intersect)

        if outerHighwayType in ['unclassified', 'pedestrian', 'cycleway']:
        
            common_nodes_nojcts = map(lambda cns: [x for x in cns if x not in jctids], common_nodes_list)
        
        else:

        # If we're looking at larger highway types (residential, primary, trunk, etc), 
        # check if any of the nodes each row has in common with outerNodes is a junction of a larger type 
        # (meaning that at least two highways of a larger type - residential, primary, trunk, etc - intersect)

            common_nodes_nojcts = map(lambda cns: [x for x in cns if x not in larger_jctids], common_nodes_list)

        # Grab the indices of all rows where the resultant list of nodes (shared with outerNodes, but not a junction)
        # isn't empty
        # 'if list' (here: 'if nodes') returns true if list is non-empty, false if list is empty

        neighbours = [ind for ind, nodes in zip(index_list, common_nodes_nojcts) if nodes]

        # Remove self
        
        neighbours_without_self = [x for x in neighbours if x != outerInd]

        return neighbours_without_self

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    ops_number = unfoldedOddballs.index.size

    # add row number, needed to check only x rows above/below current row in getNeighbours
    unfoldedOddballs['row_number'] = [x for x in range(0, ops_number)]

    neighbours_list = []
    bar = tqdm(total=ops_number, desc="Computing Neighbours")
    for node, highway_type, id, row_number in zip(unfoldedOddballs['segment_nodes_ids'], unfoldedOddballs['highwaytype'], unfoldedOddballs.index, unfoldedOddballs['h3']):
        neighbours_list.append(getNeighbours(node, highway_type, id, row_number))
        bar.update(1)

    unfoldedOddballs['neighbours'] = neighbours_list
    bar.close()

    return unfoldedOddballs

In [123]:
oddballsWithNeighbours = findNeighboursH3(segs4h3eval, aux_junctions_df)


Computing Neighbours:   2%|▏         | 38/1872 [00:00<00:04, 372.64it/s]


578
604
604
604
604
755
888
888
241
888
578
277
881
503
503
102
503
604
604
926
926
926
344
344
95
344
529
529
529
529
529
529
529
238
238
578
604
604
604
277
881
881
238
238
578
595
241
881
595
926
926
595
888
344
344
708
708
926
888
708
708
95
95
95
503
529
503
595
529
529
385
385
595
241
241
241


Computing Neighbours:   6%|▌         | 114/1872 [00:00<00:04, 363.45it/s]

529
40
241
529
926
926
926
926
881
881
888
881
529
266
552
708
241
755
552
552
552
529
503
708
708
708
578
578
578
604
604
604
529
344
344
155
604
604
604
888
881
595
595
595
595
102
503
503
40
241
385
137
881
881
341
552
926
926
926
888
888
926
926
926
888
926
926
926
128
385
552
595



Computing Neighbours:  10%|▉         | 187/1872 [00:00<00:05, 336.31it/s]

...

529
888
604
529
926
926
926
926
888
518
385
926
604
518
926
708
888
277
552
552
881
578
552
155
595
595
518
518
604
604
604
604
604
604
604
604
604
604
604
604
277
277
277
277
277
277
595
595
881
926
926
529
529
529
529
529
529
529
529
238
529
881
881
881
881



Computing Neighbours:  97%|█████████▋| 1824/1872 [00:05<00:00, 331.62it/s]

881
529
529
238
238
552
604
604
385
385
385
385
385
385
385
385
385
277
277
604
604
755
755
888
888
888
888
888
888
595
595
595
595
595
595
595
595
595
595
595
595
595
595
926
926
881
881
385
518
881
888
604
529
238
344
518
518
518
518
518
518
518
518
518
518
518
888
888
888



Computing Neighbours: 100%|██████████| 1872/1872 [00:05<00:00, 323.38it/s]

277
277
277
277
277
277
888
529
238
881
881
881
888
888
888
881
552
552
888
503


(b) Neighbour search conventional for comparison

In [124]:
def findNeighboursConventional(unfoldedOddballs, junctionsdf):

    # these are ALL junctions (i.e., intersections of at least two highways, irrespective of their type)

    jctids = junctionsdf['id'].values

    # now we also need the LARGER junctions (i.e., intersections of at least two highways of a larger type)

    larger_jcts = junctionsdf[junctionsdf['junction'] == 'large_junction']

    larger_jctids = larger_jcts['id'].values 

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Determine neighbours based on shared nodes that aren't junctions.

    def getNeighbours(outerNodes, outerHighwayType, outerInd, row_number):

        max_row_diff = 1500
        # we do not need to check all rows for neighbours. to be safe, let's check max_row_diff above and max_row_diff below row number
        lower_range = max(0, row_number - max_row_diff)
        upper_range = min(unfoldedOddballs['row_number'].max(), row_number + max_row_diff)

        interval = range(lower_range, upper_range)

        # Slice data frame according to range - this operation might be expensive too though 
        # and thus be eliminated in the future
        df_slice = unfoldedOddballs[unfoldedOddballs['row_number'].isin(interval)]

        # prepare lists outside of map operations to get rid of expensive pandas operations
        segment_nodes_list = df_slice['segment_nodes_ids'].tolist()
        index_list = df_slice.index.tolist()

        # Filter the 'segment_nodes_ids' column so that only those elements that are also contained in outerNodes
        # remain
        
        common_nodes = map(lambda innerNodes: set(innerNodes).intersection(set(outerNodes)), segment_nodes_list)

        # Convert back to list (from set)

        common_nodes_list = map(lambda x: list(x), common_nodes)

        # If we're looking at smaller highway types, check if any of the nodes each row has in common with outerNodes
        # is a junction of any type (small or large, the last one meaning that at least two highways of a larger
        # type - residential, primary, trunk, etc - intersect)

        if outerHighwayType in ['unclassified', 'pedestrian', 'cycleway']:
        
            common_nodes_nojcts = map(lambda cns: [x for x in cns if x not in jctids], common_nodes_list)
        
        else:

        # If we're looking at larger highway types (residential, primary, trunk, etc), 
        # check if any of the nodes each row has in common with outerNodes is a junction of a larger type 
        # (meaning that at least two highways of a larger type - residential, primary, trunk, etc - intersect)

            common_nodes_nojcts = map(lambda cns: [x for x in cns if x not in larger_jctids], common_nodes_list)

        # Grab the indices of all rows where the resultant list of nodes (shared with outerNodes, but not a junction)
        # isn't empty
        # 'if list' (here: 'if nodes') returns true if list is non-empty, false if list is empty

        neighbours = [ind for ind, nodes in zip(index_list, common_nodes_nojcts) if nodes]

        # Remove self
        
        neighbours_without_self = [x for x in neighbours if x != outerInd]

        return neighbours_without_self

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    ops_number = unfoldedOddballs.index.size

    # add row number, needed to check only x rows above/below current row in getNeighbours
    unfoldedOddballs['row_number'] = [x for x in range(0, ops_number)]

    neighbours_list = []
    bar = tqdm(total=ops_number, desc="Computing Neighbours")
    for node, highway_type, id, row_number in zip(unfoldedOddballs['segment_nodes_ids'], unfoldedOddballs['highwaytype'], unfoldedOddballs.index, unfoldedOddballs['row_number']):
        neighbours_list.append(getNeighbours(node, highway_type, id, row_number))
        bar.update(1)

    unfoldedOddballs['neighbours'] = neighbours_list
    bar.close()

    return unfoldedOddballs

In [125]:
oddballsWithNeighbours_conv = findNeighboursH3(segs4h3eval, aux_junctions_df)


Computing Neighbours:   2%|▏         | 29/1872 [00:00<00:06, 288.23it/s]

578
604
604
604
604
755
888
888
241
888
578
277
881
503
503
102
503
604
604
926
926
926
344
344
95
344
529
529
529
529
529



Computing Neighbours:   3%|▎         | 61/1872 [00:00<00:05, 304.54it/s]

529
529
238
238
578
604
604
604
277
881
881
238
238
578
595
241
881
595
926
926
595
888
344
344
708
708
926
888
708
708
95
95



Computing Neighbours:   5%|▍         | 92/1872 [00:00<00:05, 304.46it/s]

...

888
888
595
595
595
595
595
595
595
595
595
595
595
595
595
595
926
926
881
881
385
518
881
888
604
529
238
344
518



Computing Neighbours:  99%|█████████▊| 1844/1872 [00:07<00:00, 228.71it/s]

518
518
518
518
518
518
518
518
518
518
888
888
888
277
277
277
277
277
277
888



Computing Neighbours: 100%|██████████| 1872/1872 [00:07<00:00, 261.70it/s]

529
238
881
881
881
888
888
888
881
552
552
888
503


In [126]:
# find difference
print("Self = H3, Other = Old Strategy")
oddballsWithNeighbours["neighbours"].compare(oddballsWithNeighbours_conv["neighbours"])

Self = H3, Other = Old Strategy


,self,other
